# Price Finder Feature Extraction Model

This notebook contains the development of the CNN image classification model used in the Price Finder API to identify vehicles through images

## A quick run-through of the dataset

In [ ]:
from helper_script import walk_through_dir

# The path to the images folder that contains the train/test/validate image sets
PATH = "images"

walk_through_dir(PATH)

## Image preprocessing

Image preprocessing is done through converting the downloaded images into rgb color format since there can be instances where images of different types such as png to be automatically saved as jpg through the web scraper used, but in reality they would still contain that extra alpha channel making those images 4-channels. For the model that is developed for the Price Finder API, all the images should be converted in 3-channels. Therefore RGB

In [1]:
# A script from the helper_script is used to convert the images

from helper_script import convert_images_rgb

convert_images_rgb(PATH)

## Create the data batches

In [ ]:
train_dir = "images/train"
test_dir = "images/test"
validation_dir = "images/validate"

import tensorflow as tf

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                batch_size=BATCH_SIZE,
                                                                crop_to_aspect_ratio=False)

test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                batch_size=BATCH_SIZE,
                                                                shuffle=False,
                                                                crop_to_aspect_ratio=False)

validation_data = tf.keras.preprocessing.image_dataset_from_directory(validation_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                batch_size=BATCH_SIZE,
                                                                shuffle=False,
                                                                crop_to_aspect_ratio=False)

In [ ]:
# Checking out the class names
train_data.class_names

## Building a transfer learning feature extraction model using the Keras Functional API

In [ ]:
# 1. Create the base model with tf.keras.applications
base_model = tf.keras.applications.EfficientNetB1(include_top = False)

# 2. Freeze the base model (the underlying pre-trained patterns aren't updated during training)
base_model.trainable = False

# 3. Create inputs into our model
inputs = tf.keras.layers.Input(shape = (224, 224, 3), name = "input_layer")

# 4. Pass the inputs to the base_model
x = base_model(inputs)
print(f"Shape after passing inputs through base model: {x.shape}")

# 5. Avergae pool the outputs of the base model (aggregate all the most important information, reduce the number of computations)
x = tf.keras.layers.GlobalAveragePooling2D(name = "global_average_pooling_layer")(x)
print(f"Shape after GlobalAveragePooling2D: {x.shape}")

# 6. Create the output activation layer
outputs = tf.keras.layers.Dense(4, activation = "softmax", name = "output_layer")(x)

# 7. Combine the inputs with the outputs into a model
model_1 = tf.keras.Model(inputs, outputs)

# 8. Compile the model
model_1.compile(loss = "categorical_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

# 9. Fit the model and save its history
history_1 = model_1.fit(train_data,
                        epochs = 5,
                        steps_per_epoch = len(train_data),
                        validation_data = validation_data,
                        validation_steps = len(validation_data),
                        callbacks = [create_tensorboard_callback(dir_name = "price_finder_model",
                                                                experiment_name = "model_1_EfficientNetB1")])